As usual we're start with cleaning and attempt to delay EDA until the end.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../data/train.csv')
print(df.shape)
df.head()

#1.2 million people... no wonder this was such a big file. Realistic considering the industy and what I've worked with
#in the past.

#Recall that premium is this competion's target.

(1200000, 21)


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

In [5]:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()])

Index(['age', 'annual_income', 'marital_status', 'number_of_dependents',
       'occupation', 'health_score', 'previous_claims', 'vehicle_age',
       'credit_score', 'insurance_duration', 'customer_feedback'],
      dtype='object')


In [27]:
#After back and fourth with ChatGPT and I, I figured this approach is better. No need to loop.

thresh = .02

null_counts = pd.DataFrame({'null_sum': df.isnull().sum()})
null_counts['ratio'] = round(null_counts['null_sum']/df.shape[0], 3)
null_concerns = null_counts[null_counts['ratio']>thresh].sort_values('ratio', ascending=False)
null_concerns

#So, previous claims has quite a lot of nulls... and even occupation does. Interesting. I wonder if that's th same as unemployed.
#Credit score is always inteesting to see. Dependents are reasonable to ask, as well as feedback. Perhaps some similar issues iwth
#health score as well (ie on the surface I'd assume dubious data doesn't have a health score). No comment on income (in the
#context of nulls).

,null_sum,ratio
previous_claims,364029,0.303
occupation,358075,0.298
credit_score,137882,0.115
number_of_dependents,109672,0.091
customer_feedback,77824,0.065
health_score,74076,0.062
annual_income,44949,0.037


## Initial EDA

Note that this will be my first projec where I hopefully remember to get into the habit of pasting below into a bit of code to later preserve any changes made to the data to a cleaned csv.

In [28]:
i = -1

In [29]:
i += 1 #Eh...
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 0.


id
0          8.333333e-07
799988     8.333333e-07
800004     8.333333e-07
800003     8.333333e-07
800002     8.333333e-07
               ...     
399999     8.333333e-07
399998     8.333333e-07
399997     8.333333e-07
399996     8.333333e-07
1199999    8.333333e-07
Name: proportion, Length: 1200000, dtype: float64

In [30]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 1.


age
53.0    0.022309
61.0    0.022194
39.0    0.022045
64.0    0.022001
57.0    0.021985
43.0    0.021981
62.0    0.021882
46.0    0.021803
33.0    0.021779
47.0    0.021763
44.0    0.021707
34.0    0.021684
58.0    0.021624
31.0    0.021586
56.0    0.021544
32.0    0.021531
54.0    0.021476
38.0    0.021456
36.0    0.021451
35.0    0.021437
22.0    0.021425
59.0    0.021310
37.0    0.021301
51.0    0.021300
55.0    0.021275
49.0    0.021254
45.0    0.021249
20.0    0.021210
21.0    0.021152
50.0    0.021115
40.0    0.021059
48.0    0.021023
26.0    0.020998
24.0    0.020901
30.0    0.020895
29.0    0.020886
19.0    0.020859
42.0    0.020847
60.0    0.020819
52.0    0.020818
18.0    0.020730
28.0    0.020702
63.0    0.020556
25.0    0.020504
27.0    0.020496
41.0    0.020416
23.0    0.019661
Name: proportion, dtype: float64

In [31]:
# pd.options.display.float_format = '{:,.2f}'.format
df['age'].describe()
# pd.reset_option('display.float_format')

count    1.181295e+06
mean     4.114556e+01
std      1.353995e+01
min      1.800000e+01
25%      3.000000e+01
50%      4.100000e+01
75%      5.300000e+01
max      6.400000e+01
Name: age, dtype: float64

Banding

## Concluding Matters

In [ ]:
#As I said earlier, prepping the total changed cleanings for modeling.
#Ie kind of like on Git we have the commit before actually pushing.
df = pd.read_csv('../data/train_data.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')
for col in df.columns:
    try:
        df[col] = round(df[col], 3)
    except:
        pass